In [1]:
import os, sys
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f'Running on Python {sys.version}, Tensorflow {tf.__version__}, OpenCV {cv2.__version__}')

Running on Python 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)], Tensorflow 2.5.0, OpenCV 4.5.2


In [2]:
print('Loading models...')
prototxtPath = 'face_detector/deploy.prototxt'
weightsPath = 'face_detector/res10_300x300_ssd_iter_140000.caffemodel'
face_detector = cv2.dnn.readNet(prototxtPath, weightsPath)  # Single Shot Multi-box Detector with ResNet-10 (SSD)
print('Face detection model loaded. ')
model = load_model('MobileNet mask')
model_img_height, model_img_width = 224, 224  # The dimensions that the model is trained on
print('Face detector and mask detector models loaded.')

Loading models...
Face detection model loaded. 
Face detector and mask detector models loaded.


In [4]:
image = cv2.imread('test/2.jpg')
img_height, img_width = image.shape[:2]
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300))

In [5]:
face_detector.setInput(blob)
face_detector.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
face_detector.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
detections = face_detector.forward()

for i in range(0, detections.shape[2]):  # loop through each face detected
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:  # face detector confidence level
        # compute the coordinates of the bounding box
        box = detections[0, 0, i, 3:7] * np.array([img_width, img_height, img_width, img_height])
        (startX, startY, endX, endY) = box.astype("int")

        # ensure the bounding boxes fall within the dimensions of the frame
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(img_width - 1, endX), min(img_height - 1, endY))

        # extract the face ROI, convert it from BGR to RGB channel resize it to model dimensions, and preprocess it
        face = image[startY:endY, startX:endX]
        face = cv2.resize(cv2.cvtColor(face, cv2.COLOR_BGR2RGB), (model_img_height, model_img_width))
        face = np.expand_dims(preprocess_input(img_to_array(face)), axis=0)

        # pass the face through the model to determine if the face has a mask or not
        (noMask, Mask) = tf.nn.softmax(model.predict(face)[0])  # softmax here as model does not have due to using logit for loss

        # determine the class label and color we'll use to draw the bounding box and text
        label = "Mask On" if Mask > noMask else "No Mask"
        color = (0, 255, 0) if label == "Mask On" else (0, 0, 255)
        label = f"{label}: {max(Mask, noMask) * 100:.3f}%"  # confidence for mask detector

        # display the label and bounding box rectangle on the output frame
        cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
cv2.imshow("Result", image)
cv2.waitKey(0)  # ms to wait before closing the window, 0 is infinite
cv2.destroyAllWindows()